In [1]:
import json
import numpy as np
from random import choice
from klm.query import LM, get_unigram_probs
from code.all import get_stops
from klm.query import slor

stop_words = get_stops()

comments = []
with open("corpora/data_ccrit.spacy.jsonl", "r") as inf:
    for i in inf:
        i = json.loads(i)
        comments.append(i)

DEV = {
        'klm_model': "/Users/ahandler/research/snippetbox/klm/all_gw.binary",
        'ug_model': "/Users/ahandler/research/snippetbox/klm/all_gw.unigrams.json"
}

env = DEV
LOC = env['klm_model']
UG_MODEL = env["ug_model"]

lm = LM(loc=LOC)
up = get_unigram_probs(UG_MODEL)

In [31]:
ix = 13

import re

def print_me(ix, o):
    return o if ix == 1 else "..."

def clean_dots(str_):
    a = "[... ]{2,8}"
    b = "\.{6,}"
    fix_multi_dots = re.sub(a, "...", str_)
    remove_long_dots = re.sub(b, "...", fix_multi_dots)
    return remove_long_dots

def min_slor_scorer(input_snippet):
    ### See evernote: how do you use SLOR to make a good snippet?
    all_ = []
    for seq in input_snippet.split("..."):
        all_.append(slor(seq, lm, up))
    return min(all_)


def shorten_sentences(comment, b=100, N=1000, scorer=min_slor_scorer):

    ops = [(" ".join(comment["tokens"])[0:b],min_slor_scorer(" ".join(comment["tokens"])[0:b]))] # default answer
    
    for oo in range(N):

        included = np.ones(len(comment["tokens"]))

        for o in range(choice([1,2])):

            d1 = choice(range(len(comment["tokens"])))
            included[d1] = 0
            deleted = choice(range(10))

            for i in range(deleted):
                if i + d1 < len(included):
                    included[i + d1] = 0


        comment_snippet = [print_me(ix, o) for ix, o in zip(included, comment["tokens"])]

        sequence = " ".join([o for o in comment_snippet if o != "..."])

        remove_long_dots = clean_dots(" ".join(comment_snippet))
    
        if len(remove_long_dots) < b:
            ops.append((remove_long_dots, scorer(remove_long_dots)))

    return ops
    
    
ops = shorten_sentences(comments[ix])
ops.sort(key=lambda x:x[1], reverse=True)
print("***")
print(ops[0:4])

***
[('I think it would bring the community... represents a border between two cities', 1.7864914436959403), ('I think it would bring the community... represents a border between two cities', 1.7864914436959403), ('I think it would bring the community... represents a border between two cities', 1.7864914436959403), ('I think it would bring the community... represents... between two cities', 1.7864914436959403)]


In [32]:
lm.score("it would be", verbose=True)
slor("it would be", lm, up)

-1.6249951124191284 2: <s> it
-1.6184653043746948 3: <s> it would
-0.5090322494506836 3: it would be
-2.176880121231079 3: would be </s>


1.3191484041748047